In [68]:
import pygame
import open3d as o3d
import numpy as np
import os

### Visualize data at each timestamp

In [69]:
def get_subdirectory_names(target_dir):
    return os.listdir(target_dir)

def get_file_names(target_dir):
    return os.listdir(target_dir)

def get_timestamps(target_dir, is_sorted=True):
    timestamps_with_duplicates = [file_name.split('.')[0] for file_name in get_file_names(target_dir)]
    timestamps = list(set(timestamps_with_duplicates))
    return timestamps

def get_sorted_timestamps(target_dir):
    timestamps = get_timestamps(target_dir)
    sorted_timestamps = sorted(timestamps, key=int)
    return sorted_timestamps

SIMULATION_DATA_PATH = '/home/leppsalu/Desktop/generated_data_town02-discardable_sample'
SENSOR_DATA_DIRS = get_subdirectory_names(SIMULATION_DATA_PATH) 
SENSOR_DATA_DIRS = [dir_name for dir_name in SENSOR_DATA_DIRS if dir_name == dir_name.upper()]
LIDAR_DATA_DIRS = [dir_name for dir_name in SENSOR_DATA_DIRS if 'LIDAR' in dir_name]
RADAR_DATA_DIRS = [dir_name for dir_name in SENSOR_DATA_DIRS if 'RADAR' in dir_name]
CAMERA_DATA_DIRS = [dir_name for dir_name in SENSOR_DATA_DIRS if 'CAM_' in dir_name]
TIMESTAMPS = get_sorted_timestamps(os.path.join(SIMULATION_DATA_PATH, SENSOR_DATA_DIRS[0]))

In [70]:
i = 0
n = 100
timestamp = TIMESTAMPS[i]
n_timestamps = TIMESTAMPS[i:n]

### Visualization functions

In [71]:
def add_coordinate_frame_to_visualizer(vis, transformation_matrix, coordinate_frame_name=None, coordinate_frame_size=0.2):
    # Create an Open3D geometry from the transformation matrix
    frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=coordinate_frame_size, origin=[0, 0, 0])
    frame.transform(transformation_matrix)
    vis.add_geometry(frame)
    # If a name is provided, add a text label to the frame
    # if coordinate_frame_name is not None:
    #     text = o3d.t.geometry.TriangleMesh.create_text(coordinate_frame_name).to_legacy()
    #     text.translate([0, 0, 0.1])
    #     text.transform(transformation_matrix)

    #     vis.add_geometry(text)

def add_pointcloud_to_visualizer(vis, pointcloud, transformation_matrix=None):
    pointcloud.paint_uniform_color([1.0, 0.0, 0.0])
    if transformation_matrix is not None:
        pointcloud.transform(transformation_matrix)
    vis.add_geometry(pointcloud)

def add_all_lidar_poses_to_visualizer(vis, timestamp):
    for data_dir in LIDAR_DATA_DIRS:
        transformation_matrix = np.load(os.path.join(SIMULATION_DATA_PATH, data_dir, f'{timestamp}.npy'))
        add_coordinate_frame_to_visualizer(vis, transformation_matrix, coordinate_frame_size=3, coordinate_frame_name=data_dir)

def add_all_radar_poses_to_visualizer(vis, timestamp):
    for data_dir in RADAR_DATA_DIRS:
        transformation_matrix = np.load(os.path.join(SIMULATION_DATA_PATH, data_dir, f'{timestamp}.npy'))
        add_coordinate_frame_to_visualizer(vis, transformation_matrix, coordinate_frame_size=2, coordinate_frame_name=data_dir)

def add_all_camera_poses_to_visualizer(vis, timestamp):
    for data_dir in CAMERA_DATA_DIRS:
        transformation_matrix = np.load(os.path.join(SIMULATION_DATA_PATH, data_dir, f'{timestamp}.npy'))
        add_coordinate_frame_to_visualizer(vis, transformation_matrix, coordinate_frame_size=1.5, coordinate_frame_name=data_dir)

def add_all_sensor_poses_to_visualizer(vis, timestamp):
    for data_dir in LIDAR_DATA_DIRS:
        transformation_matrix = np.load(os.path.join(SIMULATION_DATA_PATH, data_dir, f'{timestamp}.npy'))
        add_coordinate_frame_to_visualizer(vis, transformation_matrix, coordinate_frame_size=3, coordinate_frame_name=data_dir)
    for data_dir in RADAR_DATA_DIRS:
        transformation_matrix = np.load(os.path.join(SIMULATION_DATA_PATH, data_dir, f'{timestamp}.npy'))
        add_coordinate_frame_to_visualizer(vis, transformation_matrix, coordinate_frame_size=2, coordinate_frame_name=data_dir)
    for data_dir in CAMERA_DATA_DIRS:
        transformation_matrix = np.load(os.path.join(SIMULATION_DATA_PATH, data_dir, f'{timestamp}.npy'))
        add_coordinate_frame_to_visualizer(vis, transformation_matrix, coordinate_frame_size=1.5, coordinate_frame_name=data_dir)

### Visualize lidar scans

In [ ]:
def add_lidar_readings_to_visualizer(vis, timestamp):
    for data_dir in LIDAR_DATA_DIRS:
        pointcloud = o3d.io.read_point_cloud(os.path.join(SIMULATION_DATA_PATH, data_dir, f'{timestamp}.ply'))
        transformation_matrix = np.load(os.path.join(SIMULATION_DATA_PATH, data_dir, f'{timestamp}.npy'))
        add_pointcloud_to_visualizer(vis, pointcloud, transformation_matrix)

In [73]:
# Visualize one frame
vis = o3d.visualization.Visualizer()
vis.create_window(window_name='Single frame: Ego Vehicle Sensors + LIDAR point clouds', width=1920, height=1080)

add_all_sensor_poses_to_visualizer(vis, timestamp)
add_lidar_readings_to_visualizer(vis, timestamp)

vis.run()
vis.destroy_window()

In [74]:
# Visualize multiple frames
vis = o3d.visualization.Visualizer()
vis.create_window(window_name='Multiple frames: Ego Vehicle Sensors + LIDAR point clouds', width=1920, height=1080)

for i, timestamp in enumerate(n_timestamps):
    if i % 90 == 0:
        add_all_sensor_poses_to_visualizer(vis, timestamp)
    else:
        add_all_lidar_poses_to_visualizer(vis, timestamp)
    add_lidar_readings_to_visualizer(vis, timestamp)

vis.run()
vis.destroy_window()

### Visualize radar scans

In [ ]:
def add_radar_readings_to_visualizer(vis, timestamp):
    for data_dir in RADAR_DATA_DIRS:
        pointcloud = o3d.io.read_point_cloud(os.path.join(SIMULATION_DATA_PATH, data_dir, f'{timestamp}.ply'))
        transformation_matrix = np.load(os.path.join(SIMULATION_DATA_PATH, data_dir, f'{timestamp}.npy'))
        add_pointcloud_to_visualizer(vis, pointcloud, transformation_matrix)

In [76]:
# Visualize one frame
vis = o3d.visualization.Visualizer()
vis.create_window(window_name='Single frame: Ego Vehicle Sensors + RADAR point clouds', width=1920, height=1080)

add_all_sensor_poses_to_visualizer(vis, timestamp)
add_radar_readings_to_visualizer(vis, timestamp)

vis.run()
vis.destroy_window()

In [77]:
# Visualize multiple frames
vis = o3d.visualization.Visualizer()
vis.create_window(window_name='Multiple frames: Ego Vehicle Sensors + RADAR point clouds', width=1920, height=1080)

for i, timestamp in enumerate(n_timestamps):
    if i % 90 == 0:
        add_all_sensor_poses_to_visualizer(vis, timestamp)
    else:
        add_all_lidar_poses_to_visualizer(vis, timestamp)
    add_radar_readings_to_visualizer(vis, timestamp)

vis.run()
vis.destroy_window()

### Visualize depth cam scans

In [78]:
def add_depth_camera_readings_to_visualizer(vis, timestamp):
    for data_dir in ['DEPTH_BEV']:
        pointcloud = o3d.io.read_point_cloud(os.path.join(SIMULATION_DATA_PATH, data_dir, f'{timestamp}.obstacles.ply'))
        transformation_matrix = np.load(os.path.join(SIMULATION_DATA_PATH, data_dir, f'{timestamp}.npy'))
        add_pointcloud_to_visualizer(vis, pointcloud, transformation_matrix)

In [79]:
# Visualize one frame
vis = o3d.visualization.Visualizer()
vis.create_window(window_name='Single frame: Ego Vehicle Sensors + Depth camera point clouds', width=1920, height=1080)

add_all_sensor_poses_to_visualizer(vis, timestamp)
add_depth_camera_readings_to_visualizer(vis, timestamp)

# Run the visualization
vis.run()

# Close the visualization window
vis.destroy_window()

In [80]:
# Visualize multiple frames
vis = o3d.visualization.Visualizer()
vis.create_window(window_name='Multiple frame: Ego Vehicle Sensors + Depth camera point clouds', width=1920, height=1080)

for i, timestamp in enumerate(n_timestamps):
    if i % 90 == 0:
        add_all_sensor_poses_to_visualizer(vis, timestamp)
    else:
        add_all_lidar_poses_to_visualizer(vis, timestamp)
    add_depth_camera_readings_to_visualizer(vis, timestamp)

vis.run()
vis.destroy_window()